In [91]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import loguniform
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline


In [92]:
csv_data = pd.read_csv('C:/Users/暴脾气の尧/Desktop/#TUM 2 SEMESTER/Applied Machine Intelligence/elec_forecast.csv')
#C:/Users/U388316/OneDrive - Danfoss/Desktop/usb_tum/ami/Han-20210616T220428Z-001/Han/elec_forecast.csv
csv_data.head()

,Date,Time of day,Biomass[MWh],Hydropower[MWh],Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],Other renewable[MWh],Nuclear[MWh],Fossil brown coal[MWh],...,Residual load[MWh],Lufttemp.+200cm,Lufttemp.+20cm,Bodentemp.-5cm,Bodentemp.-20cm,Luftfeuchtigkeit+200cm,Windgeschw.+250cm,Niederschlag,Globalstrahlg.+200cm,Spot price
0,2015/1/1,0,1006.25,289.75,129.25,2032.00,0.0,33.00,2677.75,3922.00,...,1380.75,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,25.02
1,2015/1/1,1,995.75,297.00,129.00,2074.25,0.0,30.75,2771.50,3830.75,...,1229.75,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,18.29
2,2015/1/1,2,1005.00,284.75,128.50,2135.25,0.0,23.50,2756.50,3704.50,...,1126.25,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,16.04
3,2015/1/1,3,1010.50,280.50,129.75,2138.25,0.0,22.00,2757.00,3518.75,...,1141.00,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,14.60
4,2015/1/1,4,1009.75,278.00,130.00,2161.00,0.0,22.00,2740.75,3529.00,...,1170.25,2.9,2.0,3.1,3.4,100.0,1.5,0.0,39.0,14.95


From linear correlation analysis, we only keep three features:
'Time of day'; 'Fossil gas[MWh]'; 'Residual load[MWh]'

In [122]:
data = csv_data[[ 'Time of day','Fossil gas[MWh]', 'Residual load[MWh]','Spot price']]
#data['Fossil gas[MWh] square'] = np.log(data[ 'Fossil gas[MWh]'])
data.head()
data2 = csv_data[['Time of day', 'Biomass[MWh]','Wind onshore[MWh]', 'Fossil gas[MWh]','Other renewable[MWh]', 'Residual load[MWh]','Spot price']]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56220 entries, 0 to 56219
Data columns (total 4 columns):
Time of day           56220 non-null int64
Fossil gas[MWh]       56124 non-null float64
Residual load[MWh]    54530 non-null float64
Spot price            56220 non-null float64
dtypes: float64(3), int64(1)
memory usage: 1.7 MB


In [6]:
data.isnull().sum()

Time of day              0
Fossil gas[MWh]         96
Residual load[MWh]    1690
Spot price               0
dtype: int64

# IMPROVEMENT 2: we can try different ways to fit missing values:

we have many fucking missing values for Residual load[MWh]

1. data1 = data.fillna(0)
2. data2 = data.fillna(data.mean())
3. data3 = data.fillna(method='bfill')
4. data4 = data.interpolate()
5. data5 = ml_fit_missing(data) # use ml to fit data, Zhuyu has tried so I dont wanna repeat
6. data6 = dl_fit_missing(data) # use dl to fit data, missed info, only adaptable to sufficient large dataset

It turns out that different methods lead to only slight change, one reason may be that the percentage of missing data is quite small


In [7]:
#dl to fit missing data
import datawig
df_train, df_test = datawig.utils.random_split(data)

#初始化一个简单的imputer模型
imputer = datawig.SimpleImputer(
    input_columns=['Time of day','Spot price'], # 我们要输入的列
    output_column= 'Fossil gas[MWh]', # 我们要为其注入值的列
    output_path = 'imputer_model' #存储模型数据和度量
    )

#拟合训练数据的模型
imputer.fit(train_df=df_train, num_epochs=10)

#输入丢失的值并返回原始的数据模型和预测
imputed = imputer.predict(df_test)
imputed.info()

2021-06-18 21:52:20,692 [INFO]  
========== start: fit model
2021-06-18 21:52:20,693 [WARNING]  Already bound, ignoring bind()
2021-06-18 21:52:22,275 [INFO]  Epoch[0] Batch [0-1265]	Speed: 13099.41 samples/sec	cross-entropy=10.726536	Fossil gas[MWh]-accuracy=0.026066
2021-06-18 21:52:23,792 [INFO]  Epoch[0] Train-cross-entropy=10.817361
2021-06-18 21:52:23,793 [INFO]  Epoch[0] Train-Fossil gas[MWh]-accuracy=0.026087
2021-06-18 21:52:23,794 [INFO]  Epoch[0] Time cost=3.095
2021-06-18 21:52:23,803 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2021-06-18 21:52:23,941 [INFO]  Epoch[0] Validation-cross-entropy=11.269368
2021-06-18 21:52:23,942 [INFO]  Epoch[0] Validation-Fossil gas[MWh]-accuracy=0.000000
2021-06-18 21:52:25,393 [INFO]  Epoch[1] Batch [0-1265]	Speed: 13986.32 samples/sec	cross-entropy=10.410601	Fossil gas[MWh]-accuracy=0.026066
2021-06-18 21:52:26,801 [INFO]  Epoch[1] Train-cross-entropy=10.596464
2021-06-18 21:52:26,802 [INFO]  Epoch[1] Train-Fossil gas[MWh

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11244 entries, 22768 to 17673
Data columns (total 5 columns):
Time of day                11244 non-null int64
Fossil gas[MWh]            11221 non-null float64
Residual load[MWh]         10912 non-null float64
Spot price                 11244 non-null float64
Fossil gas[MWh]_imputed    11244 non-null float64
dtypes: float64(4), int64(1)
memory usage: 527.1 KB


From above we can see that dl datawig can fill missing values but lost many info
so try other methods

In [123]:
#general approach to fit missing data
data = data.fillna(method='bfill')
data2 = data2.fillna(method='bfill')

In [135]:
train = data[:52595]
test = data[52596:]
X_train = train.iloc[:, :-1]
Y_train = train.iloc[:, -1:]
X_test = test.iloc[:, :-1]
Y_test = test.iloc[:, -1:]

scx = StandardScaler() #可选 https://blog.csdn.net/houhuipeng/article/details/93900759
scy = StandardScaler() 
cv = KFold(n_splits=10,shuffle=True,random_state=1) #可选 https://scikit-learn.org/0.23/modules/cross_validation.html#cross-validation-and-model-selection


# IMPROVEMENT 3: we can try different ML algorithms

In [139]:
#如果用岭回归
from sklearn.linear_model import Ridge
reg = Ridge()
space = dict()
space['regressor__regression__solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['regressor__regression__alpha'] = [1e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1,5e-1, 1,2,5,10,20,50, 100, 200]

In [76]:
#如果用随机森林
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor()
space = dict() 
space['regressor__regression__n_estimators'] = [20,50,70]
space['regressor__regression__max_depth'] = [1,3,5]
#space['regressor__regression__n_estimators'] = [10,30,50,70,100]
#space['regressor__regression__max_depth'] = [3,5,10,15,20]

In [43]:
#如果用Lasso：
from sklearn.linear_model import Lasso
reg = Lasso()
space = dict()
space['regressor__regression__selection'] = ['random', 'cyclic']
space['regressor__regression__alpha'] = [0.001, 0.01, 0.1, 1, 10, 100]

In [137]:
pipe = Pipeline([('scaler', scx), 
                 ('regression', reg)])
model = TransformedTargetRegressor(regressor=pipe, transformer=scy) #注意scx，reg，pipe，scy都可以自己设置
gs = GridSearchCV(model, space, scoring='neg_root_mean_squared_error', n_jobs=1, cv=cv, refit=True) #n_jobs=-1更快但有可能会报错
gs.fit(X_train,Y_train)
print("Best CV-RMSE: {:.2f}".format(-gs.best_score_)) #print结果
print(gs.best_params_)

Y_pred_train = gs.predict(X_train)
Y_pred_test = gs.predict(X_test)
print("Training RMSE: {:.2f}".format(np.sqrt(mean_squared_error(Y_train, Y_pred_train)))) 
print("Test RMSE: {:.2f}".format(np.sqrt(mean_squared_error(Y_test, Y_pred_test))))
print("Training R2: {:.2f}".format(r2_score(Y_train, Y_pred_train))) 
print("Test R2: {:.2f}".format(r2_score(Y_test, Y_pred_test)))

Best CV-RMSE: 10.82
{'regressor__regression__alpha': 0.001, 'regressor__regression__solver': 'sag'}
Training RMSE: 10.82
Test RMSE: 19.70
Training R2: 0.49
Test R2: -0.10


IMPROVEMENT 4: we can try different split size
IMPROVEMENT 5: we can try different random_state
IMPROVEMENT 6: we can try more precise hyperparameter

No matter how we adjust the hyperparameter, the accurary would be just around 49%, 
but once we use randomforest, the accuracy suddenly increases to 64% 
Since RF is a type of non-linear regression, this might be due to non-linear properties of data
In this notebook I used pearson coeff (linear correlation) since I wanted to use linear regression to fit data
So, next step is to try spearman coeff (non-linear correlation) to decide which features to drop
Then, utilize non-linear regression, for instance RF